# Obtain the albedo at SIGMA-A and SIGMA-B

Shunan Feng

In [36]:
import geemap
import ee
import pandas as pd
# import altair as alt
import geemap.colormaps as cm
# import numpy as np
# import os

# SIGMA-A
<img src="http://globalcryospherewatch.org/cryonet/questionnaire/files/site_map.62.png" alt="Matlab" width="300" height="300"/>

# SIGMA-B
<img src="http://globalcryospherewatch.org/cryonet/questionnaire/files/site_map.61.png" alt="Matlab" width="300" height="300"/>


In [37]:
Map = geemap.Map(basemap='HYBRID')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## MODIS

# Animation

In [62]:
roi = ee.Geometry.Polygon(
        [[-70.91269278237388, 78.10503647733918],
          [-70.91269278237388, 77.41819685694047],
          [-65.99081778237388, 77.41819685694047],
          [-65.99081778237388, 78.10503647733918]])
aoi = ee.Geometry.Point( -67.6283,  78.0517)
Map.addLayer(aoi, {}, 'SIGMA_A')
Map.centerObject(aoi, 8)    
aoi = ee.Geometry.Point( -69.0619,  77.5183)
Map.addLayer(aoi, {}, 'SIGMA_B')      

In [63]:
year_start = 2001 #  MODIS 2000-02-18T00:00:00 - Present
year_end = 2020
month_start = 1
month_end = 12

date_start = ee.Date.fromYMD(year_start, month_start, 1)
date_end = ee.Date.fromYMD(year_end, month_end, 31)
years = ee.List.sequence(year_start, year_end) # time range of years
months = ee.List.sequence(month_start, month_end)

In [64]:
# /*
# Albedo_BSA_vis Black-sky albedo for visible brodband
# Albedo_BSA_nir Black-sky albedo for NIR broadband
# Albedo_BSA_shortwave Black-sky albedo for shortwave broadband
# replace BSA with WSA for white scky albedo
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD43A3
# */
colFilter = ee.Filter.And(
    # ee.Filter.bounds(aoi),
    # ee.Filter.intersects('.geo', aoi),
    ee.Filter.geometry(roi),
    ee.Filter.date(date_start, date_end)
)

# MOD10A1.006 Terra Snow Cover Daily Global 500m
dataset = ee.ImageCollection('MODIS/006/MOD10A1').filter(colFilter).select('Snow_Albedo_Daily_Tile')

# dataset = ee.ImageCollection('MODIS/006/MOD10A1') \
#             .select('Snow_Albedo_Daily_Tile') \
#             .filterDate(date_start, date_end)

def funcY(y):
    def funcM(m):
        vi = dataset.filter(ee.Filter.calendarRange(y, y, 'year')) \
                    .filter(ee.Filter.calendarRange(m, m, 'month')) \
                    .median().clip(roi).divide(100) 
        return vi.set('year', y) \
                 .set('month', m) \
                 .set('system:time_start', ee.Date.fromYMD(y, m, 1).millis())
    return months.map(funcM)

albedoMonthly = ee.ImageCollection.fromImages(years.map(funcY).flatten())

In [65]:
dates = albedoMonthly.reduceColumns(ee.Reducer.toList(), ["system:time_start"]).get('list')
dates = pd.to_datetime(dates.getInfo(), unit='ms')
dates = pd.DataFrame(dates,columns =['time'])
dates = dates.time.dt.strftime('%Y-%m-%d')

In [66]:
palette = cm.get_palette('viridis', n_class=10)
visParams = {'min': 0,  'max': 1, 'palette': palette}
Map.add_time_slider(albedoMonthly, visParams, labels=dates.to_list() , time_interval=1)

In [67]:
# Multisat

In [68]:


# # aoi = ee.Geometry.Point([dfsYear.lon[i], dfsYear.lat[i]]).buffer(300)
# # aoi = ee.Geometry.Point( -67.6283,  78.0517)
# # Map.addLayer(aoi, {}, 'SIGMA_A')
# # aoi = ee.FeatureCollection([
# #     ee.Feature(ee.Geometry.Point(
# #         [
# #             dfsYear.lon[i], dfsYear.lat[i]
# #         ]
# #     ), {'label': dfsYear.Year[i]})
# # ])
# # date_start = str(dfsYear.Year[i]) + '-' + str(1) + '-' + str(1) 
# # date_end = str(dfsYear.Year[i]) + '-' + str(12) + '-' + str(31) 
# # print(date_start)
# # create filter for image collection

# colFilter = ee.Filter.And(
#     # ee.Filter.bounds(aoi),
#     # ee.Filter.intersects('.geo', aoi),
#     ee.Filter.geometry(roi),
#     # ee.Filter.date(date_start, date_end),
#     # ee.Filter.calendarRange(5, 9, 'month'),
#     ee.Filter.lt('CLOUD_COVER', 50),
#     ee.Filter.lte('GEOMETRIC_RMSE_MODEL', 30),
#     # ee.Filter.gt('SUN_ELEVATION', 5),
#     ee.Filter.Or(
#         ee.Filter.eq('IMAGE_QUALITY', 9),
#         ee.Filter.eq('IMAGE_QUALITY_OLI', 9)
#     )
# )

# s2colFilter =  ee.Filter.And(
#     # ee.Filter.bounds(aoi),
#     # ee.Filter.intersects('.geo', aoi),
#     ee.Filter.geometry(roi),
#     # ee.Filter.date(date_start, date_end),
#     # ee.Filter.calendarRange(5, 9, 'month'),
#     ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)
# )

# # oliCol = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
# oliCol = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
#             .filter(colFilter) \
#             .map(prepOli) \
#             .select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])
# # etmCol = ee.ImageCollection("LANDSAT/LE07/C01/T1_TOA")
# etmCol = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
#             .filter(colFilter) \
#             .map(prepEtm) \
#             .select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])
# tmCol = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
#             .filter(colFilter) \
#             .map(prepEtm) \
#             .select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])
# s2Col = ee.ImageCollection('COPERNICUS/S2_SR') \
#             .filter(s2colFilter) \
#             .map(prepS2) \
#             .select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])
# # landsatCol = etmCol.merge(tmCol)
# landsatCol = oliCol.merge(etmCol).merge(tmCol)
# multiSat = landsatCol.merge(s2Col).sort('system:time_start')

# # if multiSat.size().getInfo()==0:
# #     continue
# def addAlbedo(image):
#     # imdate = image.get('system:time_start')
#     albedo = image.expression(
#         '0.356 * b2/10000 + 0.13 * b4/10000 + 0.373 * b5/10000 + 0.085 * b6/10000 + 0.072 * b7/10000 - 0.0018',
#         {
#             'b2': image.select('Blue'),
#             # 'b3': image.select('Red'),
#             'b4': image.select('Red'),
#             'b5': image.select('NIR'),
#             'b6': image.select('SWIR1'),
#             'b7': image.select('SWIR2'),
#             # b8: image.select('B8'),
#             # b11: image.select('B11'),
#             # b12: image.select('B12')
#         }
#     ).rename('albedo')
#     return image.addBands(albedo).copyProperties(image, ['system:time_start'])
# multiSatAlbedo = multiSat.map(addAlbedo)

In [69]:
# dates = multiSatAlbedo.reduceColumns(ee.Reducer.toList(), ["system:time_start"]).get('list')
# dates = pd.to_datetime(dates.getInfo(), unit='ms')
# dates = pd.DataFrame(dates,columns =['time'])
# dates = dates.time.dt.strftime('%Y-%m-%d')


# palette = cm.get_palette('viridis', n_class=10)
# visParams = {'min': 0,  'max': 1, 'palette': palette}
# Map.add_time_slider(multiSatAlbedo.select('albedo'), visParams, labels=dates.to_list() , time_interval=1)

In [70]:
# Map.add_time_slider(multiSatAlbedo.select('albedo'), visParams, labels=dates.to_list() , time_interval=1)

In [73]:
Map.add_colorbar(visParams, label="albedo", orientation="horizontal", layer_name="albedo")